In [95]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
import sklearn.metrics as skm
import pandas as pd
import numpy as np

data_train = pd.read_csv("../preprocessing/preprocesed_variables_train.csv")
X = data_train.drop(columns='health')
y = data_train['health']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=20712124, stratify = y)

,year,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x1196,x1197,x1198,x1199,x1200,x1201,x1202,x1203,x1204,x1205
15700,1.000000,0.4,0.2,0.6,0.8,0.0,0.0,0.444444,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.933333,0.857143,0.777778,1.0
5554,0.333333,0.4,0.6,0.4,0.4,1.0,1.0,0.333333,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.866667,0.142857,0.444444,0.0
11976,0.666667,0.8,0.6,0.4,0.4,0.0,0.0,0.444444,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.400000,0.285714,0.222222,0.0
13812,0.833333,0.6,0.8,0.2,0.4,0.0,1.0,0.666667,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.933333,0.285714,0.444444,0.0
12291,0.666667,0.4,0.6,0.6,0.4,0.0,0.0,0.888889,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.533333,0.285714,0.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11432,0.666667,0.4,0.6,0.4,0.4,0.0,0.0,0.222222,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.866667,0.714286,0.111111,0.0
8461,0.500000,0.4,0.6,0.8,0.8,0.0,0.0,0.555556,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.466667,0.285714,0.222222,0.0
6188,0.333333,0.4,0.2,0.4,0.4,1.0,1.0,0.555556,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.673333,1.000000,1.000000,0.0
5895,0.333333,0.4,0.2,0.4,0.4,1.0,1.0,0.666667,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.400000,0.142857,0.000000,0.0


In [96]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 500))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(n_estimators=500))

In [97]:
selected_feat= X_train.columns[(sel.get_support())]
#len(selected_feat)
#print(selected_feat)

#pd.Series(sel.estimator_.feature_importances_.ravel()).hist()
selected_feat

Index(['x1', 'x2', 'x3', 'x4', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12',
       ...
       'x1154', 'x1157', 'x1175', 'x1178', 'x1179', 'x1185', 'x1202', 'x1203',
       'x1204', 'x1205'],
      dtype='object', length=235)

In [99]:
from sklearn.model_selection import GridSearchCV

knn_cv = KNeighborsClassifier()

#create a dictionary of all values we want to test for n_neighbors
param_grid = {'n_neighbors': np.arange(1, 150, 5)}

#use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(knn_cv, param_grid, cv = 5, verbose = 2)

knn_gscv.fit(X_train[selected_feat], y_train)

knn_gscv.best_params_, knn_gscv.best_score_

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END ......................................n_neighbors=1; total time=   0.4s
[CV] END ......................................n_neighbors=1; total time=   0.3s
[CV] END ......................................n_neighbors=1; total time=   0.3s
[CV] END ......................................n_neighbors=1; total time=   0.3s
[CV] END ......................................n_neighbors=1; total time=   0.3s
[CV] END ......................................n_neighbors=6; total time=   0.5s
[CV] END ......................................n_neighbors=6; total time=   0.5s
[CV] END ......................................n_neighbors=6; total time=   0.5s
[CV] END ......................................n_neighbors=6; total time=   0.5s
[CV] END ......................................n_neighbors=6; total time=   0.5s
[CV] END .....................................n_neighbors=11; total time=   0.5s
[CV] END .....................................n

({'n_neighbors': 131}, 0.4480523426303377)

In [101]:
knn = KNeighborsClassifier(n_neighbors = 130)
knn.fit(X_train[selected_feat], y_train)
y_pred = knn.predict(X_test[selected_feat])
y_pred_proba = knn.predict_proba(X_test[selected_feat])

skm.log_loss(y_test, y_pred_proba)
skm.accuracy_score(y_test, y_pred)

0.4471264367816092

In [104]:
pd.DataFrame(selected_feat).to_csv("important.csv", index = False)